# Example template

Download this notebook from:
https://github.com/qcodespp/qcodespp.github.io/blob/main/docs/source/qcodespp_template.ipynb

## Initialise

In [ ]:
# Optional: check for VISA instruments

import qcodespp as qc
qc.listVISAinstruments()

Instrument IDN: KEITHLEY INSTRUMENTS,MODEL 2450,04613345,1.7.12b
 VISA Address: TCPIP0::169.254.100.202::inst0::INSTR

Cannot access instrument with address ASRL1::INSTR. Likely the instrument is already connected, possibly in another program or ipython kernel.

Cannot access instrument with address ASRL3::INSTR. Likely the instrument is already connected, possibly in another program or ipython kernel.

Instrument IDN: QDevil, QDAC-II, 159, 13-1.57
 VISA Address: ASRL9::INSTR



In [ ]:
# Import packages, instrument drivers, and connect to instruments.

import qcodespp as qc
import numpy as np
import time

from qcodespp.instrument_drivers.QDevil.QDAC2 import QDac2
from qcodespp.instrument_drivers.ZI.ZIHF2LI import ZIHF2LI
from qcodespp.instrument_drivers.tektronix.Keithley_2450 import Keithley_2450

qdac = QDac2('qdac','ASRL9::INSTR')
li = ZIHF2LI('li','dev1832') # ZI instruments don't use VISA
k2450 = Keithley_2450('k2450','TCPIP0::169.254.100.202::inst0::INSTR')

Connected to: QDevil QDAC-II (serial:159, firmware:13-1.57) in 0.83s
Connected to: Zurich Instruments HF2LI (serial:dev1832, hardware:6) as li in 0.04s
Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04613345, firmware:1.7.12b) in 0.02s


In [ ]:
# Define custom parameters

stepper=qc.Parameter('stepper',label='Stepper',unit='',set_cmd=lambda val: 0)

currentX=qc.ScaledParameter(li.demod0_X,gain=1e-6,name='currentX',label='Current',unit='A')
currentY=qc.ScaledParameter(li.demod0_Y,gain=1e-6,name='currentY',label='Current',unit='A')
voltageX=qc.ScaledParameter(li.demod1_X,gain=1e-3,name='voltageX',label='Voltage',unit='V')
voltageY=qc.ScaledParameter(li.demod1_Y,gain=1e-3,name='voltageY',label='Voltage',unit='V')

resistance=qc.Parameter('resistance',label='Resistance',unit='Ohm',get_cmd=lambda: voltageX()/currentX())

# Tell qcodespp where to store data
qc.set_data_folder('data')

# Init the station.
station=qc.Station(add_variables=globals())

Automatically adding components to Station...
Instruments in station: qdac, li, k2450
Parameters in station: timer, stepper, currentX, currentY, voltageX, voltageY, resistance


## 1D measurements

In [ ]:
# 1D loops
loop=qc.loop1d(sweep_parameter=qdac.ch01.volt,
               start=0,stop=1,num=101,delay=0.03,
               device_info='Device1',
               instrument_info=f'ACdiv=1e5 DCdiv=1e3 freq={li.osc0_freq():.6g} Hz',
               measure=[currentX, currentY, voltageX, voltageY, resistance],
               plot=[currentX,voltageX,resistance])

Are you sure? Start value for volt.sweep is 0 V but volt()=0.9999981 V
python C:\git\qcodespp\qcodespp\plotting\RemotePlotClient.py qcodes.plot.899e7ccf0ec842c3866a64ac0ad26aa2 8878 9156
DataSetPP:
   location = 'data/#045_Device1 qdac_ch01_volt(0 1)V with ACdiv=1e5 DCdiv=1e3 freq=4e+07 Hz_2025-06-13_16-00-00'
   <Type>   | <array_id>         | <array.shape>
   Setpoint | qdac_ch01_volt_set | (101,)
   Measured | currentX           | (101,)
   Measured | currentY           | (101,)
   Measured | voltageX           | (101,)
   Measured | voltageY           | (101,)
   Measured | resistance         | (101,)
   Measured | timer              | (101,) 
Estimated time: 12.899286870956422 s, 0.21498811451594035 mins, 0.0035831352419323395 hours
Done at: Fri Jun 13 16:00:16 2025


In [ ]:
data=loop.run()

In [ ]:
# 1D loop with a break condition
loop=qc.loop1d(sweep_parameter=k2450.volt,
               start=0,stop=1,num=101,delay=0.03,
               device_info='Device1',
               instrument_info=f'ACdiv=1e5 DCdiv=1e3 freq={li.osc0_freq():.6g} Hz',
               measure=[qc.BreakIf(lambda: np.abs(k2450.curr())>1e-9), 
                        currentX, currentY, voltageX, voltageY, resistance],
               plot=[currentX,voltageX,resistance])

Are you sure? Start value for volt.sweep is 0 V but volt()=1.0 V
python C:\git\qcodespp\qcodespp\plotting\RemotePlotClient.py qcodes.plot.126c8cd213d34a43be7995b7e843b34b 8878 9188
DataSetPP:
   location = 'data/#045_Device1 k2450_volt(0 1)V with ACdiv=1e5 DCdiv=1e3 freq=4e+07 Hz_2025-06-13_16-00-03'
   <Type>   | <array_id>     | <array.shape>
   Setpoint | k2450_volt_set | (101,)
   Measured | currentX       | (101,)
   Measured | currentY       | (101,)
   Measured | voltageX       | (101,)
   Measured | voltageY       | (101,)
   Measured | resistance     | (101,)
   Measured | timer          | (101,) 
Estimated time: 21.359037165641784 s, 0.3559839527606964 mins, 0.00593306587934494 hours
Done at: Fri Jun 13 16:00:28 2025


In [6]:
data=loop.run()

Started at 2025-06-13 16:00:07


  0%|          | 0/101 [00:00<?, ?it/s]. Estimated finish time: 1970-01-01 00:00:00+00:00

DataSetPP:
   location = 'data/#045_Device1 k2450_volt(0 1)V with ACdiv=1e5 DCdiv=1e3 freq=4e+07 Hz_2025-06-13_16-00-03'
   <Type>   | <array_id>     | <array.shape>
   Setpoint | k2450_volt_set | (101,)
   Measured | currentX       | (101,)
   Measured | currentY       | (101,)
   Measured | voltageX       | (101,)
   Measured | voltageY       | (101,)
   Measured | resistance     | (101,)
   Measured | timer          | (101,)
Finished at 2025-06-13 16:01:12


## 2D measurements

In [ ]:
# 2D loop
loop=qc.loop2d(sweep_parameter=qdac.ch01.volt,
                start=0,stop=1,num=11,delay=0.03,
                step_parameter=k2450.volt,
                step_start=0,step_stop=1,step_num=11,step_delay=0.1,
                device_info='Device1',
                instrument_info=f'ACdiv=1e5 DCdiv=1e3 freq={li.osc0_freq():.6g} Hz',
                measure=[currentX, currentY, voltageX, voltageY, resistance],
                plot=[currentX,voltageX,resistance])

Are you sure? Start value for volt.sweep is 0 V but volt()=0.9999981 V
Are you sure? Start value for volt.sweep is 0 V but volt()=1.0 V
python C:\git\qcodespp\qcodespp\plotting\RemotePlotClient.py qcodes.plot.a45bf8a7e7684a84a80eab44fd98e74d 8878 9183
DataSetPP:
   location = 'data/#046_Device1 k2450_volt(0 1)V qdac_ch01_volt(0 1)V with ACdiv=1e5 DCdiv=1e3 freq=4e+07 Hz_2025-06-13_16-01-12'
   <Type>   | <array_id>         | <array.shape>
   Setpoint | k2450_volt_set     | (11,)
   Setpoint | qdac_ch01_volt_set | (11, 11)
   Measured | currentX           | (11, 11)
   Measured | currentY           | (11, 11)
   Measured | voltageX           | (11, 11)
   Measured | voltageY           | (11, 11)
   Measured | resistance         | (11, 11)
   Measured | timer              | (11, 11) 
Estimated time: 17.788989906311034 s, 0.2964831651051839 mins, 0.004941386085086399 hours
Done at: Fri Jun 13 16:01:33 2025


In [8]:
data=loop.run()

Started at 2025-06-13 16:01:15


  0%|          | 0/11 [00:00<?, ?it/s]. Estimated finish time: 1970-01-01 00:00:00+00:00

DataSetPP:
   location = 'data/#046_Device1 k2450_volt(0 1)V qdac_ch01_volt(0 1)V with ACdiv=1e5 DCdiv=1e3 freq=4e+07 Hz_2025-06-13_16-01-12'
   <Type>   | <array_id>         | <array.shape>
   Setpoint | k2450_volt_set     | (11,)
   Setpoint | qdac_ch01_volt_set | (11, 11)
   Measured | currentX           | (11, 11)
   Measured | currentY           | (11, 11)
   Measured | voltageX           | (11, 11)
   Measured | voltageY           | (11, 11)
   Measured | resistance         | (11, 11)
   Measured | timer              | (11, 11)
Finished at 2025-06-13 16:01:31


In [ ]:
# 2D loops with sweep in both directions.
loop=qc.loop2dUD(sweep_parameter=qdac.ch01.volt,
                start=0,stop=1,num=11,delay=0.03,
                step_parameter=k2450.volt,
                step_start=0,step_stop=1,step_num=11,step_delay=0.1,
                device_info='Device1',
                instrument_info=f'ACdiv=1e5 DCdiv=1e3 freq={li.osc0_freq():.6g} Hz',
                measure=[currentX, currentY, voltageX, voltageY, resistance],
                plot=[currentX,voltageX,resistance])

Are you sure? Start value for volt.sweep is 0 V but volt()=0.9999981 V
Are you sure? Start value for volt.sweep is 0 V but volt()=1.0 V
python C:\git\qcodespp\qcodespp\plotting\RemotePlotClient.py qcodes.plot.36de52595d6240f5abb9ce9ae11e9055 8878 9123
DataSetPP:
   location = 'data/#047_Device1 k2450_volt(0 1)V qdac_ch01_volt(0 1)V with ACdiv=1e5 DCdiv=1e3 freq=4e+07 Hz_2025-06-13_16-01-31'
   <Type>   | <array_id>           | <array.shape>
   Setpoint | k2450_volt_set       | (11,)
   Setpoint | qdac_ch01_volt_set_0 | (11, 11)
   Measured | currentX_0           | (11, 11)
   Measured | currentY_0           | (11, 11)
   Measured | voltageX_0           | (11, 11)
   Measured | voltageY_0           | (11, 11)
   Measured | resistance_0         | (11, 11)
   Measured | timer_0              | (11, 11)
   Setpoint | qdac_ch01_volt_set_1 | (11, 11)
   Measured | currentX_1           | (11, 11)
   Measured | currentY_1           | (11, 11)
   Measured | voltageX_1           | (11, 11)
   Mea

In [10]:
data=loop.run()

Started at 2025-06-13 16:01:34


  0%|          | 0/11 [00:00<?, ?it/s]. Estimated finish time: 1970-01-01 00:00:00+00:00

DataSetPP:
   location = 'data/#047_Device1 k2450_volt(0 1)V qdac_ch01_volt(0 1)V with ACdiv=1e5 DCdiv=1e3 freq=4e+07 Hz_2025-06-13_16-01-31'
   <Type>   | <array_id>           | <array.shape>
   Setpoint | k2450_volt_set       | (11,)
   Setpoint | qdac_ch01_volt_set_0 | (11, 11)
   Measured | currentX_0           | (11, 11)
   Measured | currentY_0           | (11, 11)
   Measured | voltageX_0           | (11, 11)
   Measured | voltageY_0           | (11, 11)
   Measured | resistance_0         | (11, 11)
   Measured | timer_0              | (11, 11)
   Setpoint | qdac_ch01_volt_set_1 | (11, 11)
   Measured | currentX_1           | (11, 11)
   Measured | currentY_1           | (11, 11)
   Measured | voltageX_1           | (11, 11)
   Measured | voltageY_1           | (11, 11)
   Measured | resistance_1         | (11, 11)
   Measured | timer_1              | (11, 11)
Finished at 2025-06-13 16:02:05
